Least Square Monte Carlo

Problem 1

Set-up

In [1]:
import numpy as np
np.random.seed(123)


def Laguerre(x, k):
    x = np.array(x) 
    L1 = np.exp(- x/2.)
    L2 = np.exp(- x/2.) * (1-x)
    L3 = np.exp(- x/2.) * (1 - 2 * x + 0.5 * x**2)
    L4 = np.exp(- x/2.) * (1 - 3 * x + 1.5 * x**2 - 1/6. * x**3)
    L = [np.stack((L1, L2), axis = -1),
         np.stack((L1, L2, L3), axis = -1),
         np.stack((L1, L2, L3, L4), axis = -1)]
    return L[k-2]

def Hermite(x, k):
    L1 = np.power(x, 0)
    L2 = 2 * np.power(x, 1)
    L3 = 4 * np.power(x, 2) - 2
    L4 = 8 * np.power(x, 3) -12 * np.array(x)
    L = [np.stack((L1, L2), axis = -1),
         np.stack((L1, L2, L3), axis = -1),
         np.stack((L1, L2, L3, L4), axis = -1)]
    return L[k-2]

def Monomials(x, k):
    L1 = np.power(x, 0)
    L2 = np.power(x, 1)
    L3 = np.power(x, 2)
    L4 = np.power(x, 3)
    L = [np.stack((L1, L2), axis = -1),
         np.stack((L1, L2, L3), axis = -1),
         np.stack((L1, L2, L3, L4), axis = -1)]
    return L[k-2]


In [4]:
def AmericanOptionsLSMC(S0, strike, T, r, sigma, simulations, k, method):

        
    N = 50
    time_unit = T / float(N)
    discount = np.exp(-r * time_unit)
    
   
    MCprice_matrix = np.zeros((simulations, N + 1), dtype=np.float64)
    MCprice_matrix[:, 0] = S0
    for t in range(1, N + 1):
        brownian = np.random.standard_normal(int(simulations / 2))
        brownian = np.concatenate((brownian, -brownian))
        MCprice_matrix[:, t] = (MCprice_matrix[:, t-1]
                              * np.exp((r - sigma ** 2 / 2.) * time_unit
                              + sigma * brownian * np.sqrt(time_unit)))   
    
    MCpayoff = np.maximum(strike - MCprice_matrix, 0)
    
      
    value_matrix = np.zeros_like(MCpayoff)
    value_matrix[:, -1] = MCpayoff[:, -1]
    index_matrix = np.zeros_like(MCpayoff)
    index_matrix[:,-1] = (value_matrix[:, -1] > 0) * 1
    for t in range(N - 1, 0 , -1):
                  
        itm = np.where(MCpayoff[:, t] > 0)
        P = MCprice_matrix[itm, t][0]         
        
        if method == "H":
            L = Hermite(P, k)
        
        elif method == "L":
            L = Laguerre(P, k)
        
        else:  
            L = Monomials(P, k)
             
        A = L.transpose() @ L
        disc = np.array([pow(discount, i+1) for i in range(N - t)])
        disc_MCpayoff = np.multiply(MCpayoff[itm, t+1:][0], disc[np.newaxis, :])
        Y = np.sum(disc_MCpayoff * index_matrix[itm, t+1:][0], axis = 1)
        b = L.transpose() @ Y
        a = np.linalg.pinv(A) @ b
        
        continuation_value = L @ a
        value_matrix[itm, t] = np.where(MCpayoff[itm, t][0] > continuation_value, MCpayoff[itm, t][0], 0)
        index_matrix[itm, t] = (value_matrix[itm, t] > 0) * 1
        exercise = np.where(index_matrix[:,t] == 1)
        index_matrix[exercise, t+1:] = 0 
        
            
    disc_vector = [pow(discount, i+1) for i in range(N)]
    index_matrix = np.multiply(index_matrix[:,1:], disc_vector)
    optvalue = np.sum(index_matrix * value_matrix[:,1:], axis = 1)   
       
   
    return sum(optvalue)/float(simulations)
    
 

a.

In [5]:

S0_1 = [36., 40., 44.]
T_1 = [0.5, 1, 2]
k_1 = [2, 3, 4]


# (a)

print("Laguerre polynomials: ")
for i in S0_1:
    for j in T_1:
        for k in k_1:
            price = AmericanOptionsLSMC(S0 = i, strike = 40., T = j, r = 0.06, sigma = 0.2, 
                                        simulations = 100000, k = k, method = 'L')
            print(str([i, j, k]) + ": " + str(price))

Laguerre polynomials: 
[36.0, 0.5, 2]: 3.98034094541
[36.0, 0.5, 3]: 4.05936982693
[36.0, 0.5, 4]: 4.18044699422
[36.0, 1, 2]: 3.96514854543
[36.0, 1, 3]: 4.08676297642
[36.0, 1, 4]: 4.27698408224
[36.0, 2, 2]: 3.91276128558
[36.0, 2, 3]: 4.11919916641
[36.0, 2, 4]: 4.33238216215
[40.0, 0.5, 2]: 1.40681532782
[40.0, 0.5, 3]: 1.70200860695
[40.0, 0.5, 4]: 1.78879255327
[40.0, 1, 2]: 1.52249826311
[40.0, 1, 3]: 1.84810497146
[40.0, 1, 4]: 2.15545293264
[40.0, 2, 2]: 1.63799006529
[40.0, 2, 3]: 2.0002242213
[40.0, 2, 4]: 2.31454365309
[44.0, 0.5, 2]: 0.477240051957
[44.0, 0.5, 3]: 0.615975067277
[44.0, 0.5, 4]: 0.626656334406
[44.0, 1, 2]: 0.645326434325
[44.0, 1, 3]: 0.877246122671
[44.0, 1, 4]: 1.07576020245
[44.0, 2, 2]: 0.817834417153
[44.0, 2, 3]: 1.04549899797
[44.0, 2, 4]: 1.33948856139


b.

In [6]:
print("Hermite polynomials: ")
for i in S0_1:
    for j in T_1:
        for k in k_1:
            price = AmericanOptionsLSMC(S0 = i, strike = 40., T = j, r = 0.06, sigma = 0.2, 
                                        simulations = 100000, k = k, method = 'H')
            print(str([i, j, k]) + ": " + str(price))
                                    

Hermite polynomials: 
[36.0, 0.5, 2]: 4.15310534426
[36.0, 0.5, 3]: 4.20265089016
[36.0, 0.5, 4]: 4.20499138819
[36.0, 1, 2]: 4.41623986736
[36.0, 1, 3]: 4.47247955174
[36.0, 1, 4]: 4.48154012384
[36.0, 2, 2]: 4.74639600823
[36.0, 2, 3]: 4.81257572657
[36.0, 2, 4]: 4.83229871271
[40.0, 0.5, 2]: 1.7680770762
[40.0, 0.5, 3]: 1.78427758664
[40.0, 0.5, 4]: 1.78220953064
[40.0, 1, 2]: 2.27550765293
[40.0, 1, 3]: 2.30344266139
[40.0, 1, 4]: 2.31858712752
[40.0, 2, 2]: 2.80990223031
[40.0, 2, 3]: 2.87152129102
[40.0, 2, 4]: 2.87785914638
[44.0, 0.5, 2]: 0.62464611395
[44.0, 0.5, 3]: 0.629091081233
[44.0, 0.5, 4]: 0.629475771673
[44.0, 1, 2]: 1.09196185736
[44.0, 1, 3]: 1.10239798944
[44.0, 1, 4]: 1.11845687964
[44.0, 2, 2]: 1.64189855199
[44.0, 2, 3]: 1.67749277107
[44.0, 2, 4]: 1.69237811024


c.

In [7]:
print("Simple Monomials ")
for i in S0_1:
    for j in T_1:
        for k in k_1:
            price = AmericanOptionsLSMC(S0 = i, strike = 40., T = j, r = 0.06, sigma = 0.2, 
                                        simulations = 100000, k = k, method = 'M')
            print(str([i, j, k]) + ": " + str(price))                          

Simple Monomials 
[36.0, 0.5, 2]: 4.16775459323
[36.0, 0.5, 3]: 4.19670555573
[36.0, 0.5, 4]: 4.1995053024
[36.0, 1, 2]: 4.42515862754
[36.0, 1, 3]: 4.45959327088
[36.0, 1, 4]: 4.47647826381
[36.0, 2, 2]: 4.74833340124
[36.0, 2, 3]: 4.82217151678
[36.0, 2, 4]: 4.8337045783
[40.0, 0.5, 2]: 1.77195950555
[40.0, 0.5, 3]: 1.78589430081
[40.0, 0.5, 4]: 1.78893469034
[40.0, 1, 2]: 2.28269662198
[40.0, 1, 3]: 2.30721372083
[40.0, 1, 4]: 2.30514229085
[40.0, 2, 2]: 2.81596166159
[40.0, 2, 3]: 2.87760600322
[40.0, 2, 4]: 2.87171076628
[44.0, 0.5, 2]: 0.628951860713
[44.0, 0.5, 3]: 0.629669984143
[44.0, 0.5, 4]: 0.627308719664
[44.0, 1, 2]: 1.08720822746
[44.0, 1, 3]: 1.10689492033
[44.0, 1, 4]: 1.10807329206
[44.0, 2, 2]: 1.66267864988
[44.0, 2, 3]: 1.67792142337
[44.0, 2, 4]: 1.67612792356


Comment:
Holding S0 and T constant, the precision of LSMC pricing increases as k increases. Allowing for more k terms enhances the performance at the cost of slower computation speed. Comparing the three method, we found that Laguerre method generates larger deviation from the true value of options. The results from Hermite and simple monomials are quite similiar while the latter one has better performance. 

Problem 2

a.

In [3]:
T = 1
t = 0.2
sigma = 0.2
r = 0.06
S0 = 65
simulations = 10000
N = 100
time_unit = T/float(N)
discount = np.exp(-r * time_unit)
price_matrix = np.zeros((simulations, N + 1), dtype=np.float64)
price_matrix[:, 0] = S0
for i in range(1, N + 1):
    brownian = np.random.standard_normal(simulations)
    price_matrix[:, i] = (price_matrix[:, i-1]
                              * np.exp((r - sigma ** 2 / 2.) * time_unit
                              + sigma * brownian * np.sqrt(time_unit)))

forward_t = int(0.2/float(time_unit))
strike = price_matrix[:, forward_t]
  
payoff_matrix = np.maximum(np.array([list(strike),] * (N+1)).transpose() - price_matrix, np.zeros_like(price_matrix))
P_2a = np.exp(- r * T) * payoff_matrix[:,-1].sum() / float(simulations) 
print("Forward-start European put: " + str(P_2a))

Forward-start European put: 3.21760923317


b.

In [4]:
M = N - forward_t
price_scale = np.divide(price_matrix[:, forward_t:], strike[:,None])
strike_scale = price_scale[:,0]
payoff_scale = np.maximum(1 - price_scale, 0)
value_matrix = np.zeros_like(payoff_scale)
value_matrix[:, -1] = payoff_scale[:, -1]
index_matrix = np.zeros_like(payoff_scale)
index_matrix[:,-1] = (value_matrix[:, -1] > 0.) * 1
for j in range(M - 1, 0 , -1):
              
    itm = np.where(payoff_scale[:, j] > 0.)
    P = price_scale[itm, j][0]         
    L = Monomials(P, 4) 
    A = L.transpose() @ L
    disc = np.array([pow(discount, i+1) for i in range(M - j)])
    disc_MCpayoff = np.multiply(payoff_scale[itm, j+1:][0], disc[np.newaxis, :])
    
    Y = np.sum(disc_MCpayoff * index_matrix[itm, j+1:][0], axis = 1)
    b = L.transpose() @ Y
    a = np.linalg.pinv(A) @ b
    
    continuation_value = L @ a
    value_matrix[itm, j] = np.where(payoff_scale[itm, j][0] > continuation_value, payoff_scale[itm, j][0], 0)
    index_matrix[itm, j] = (value_matrix[itm, j] > 0) * 1
    exercise = np.where(index_matrix[:,j] == 1)
    index_matrix[exercise, j+1:] = 0 

disc_vector = [pow(discount, i+1) for i in range(M)]
index_matrix = np.multiply(index_matrix[:,1:], disc_vector)
optvalue = np.sum(index_matrix * value_matrix[:,1:], axis = 1) * strike
P_2b = optvalue.sum()/simulations * np.exp(-r * t)
print("Forward-start American put: " + str(P_2b))

Forward-start American put: 3.51474525865


To price American forward start option, firstly we need to scale the $payoff = S_t * max(1 - S_T/S_t) $. Then we compute the scaled American value till t=0.2, unscale the value and then discount it back to t=0.